In [67]:
import re
import pandas as pd
import itertools
import random
import json
from collections import Counter


In [68]:
dataset = []
with open('../data/ViMedNER.txt', 'r', encoding='utf-8') as f:
    for dong in f:
        dataset.append(json.loads(dong.strip()))

In [69]:
df = pd.DataFrame(dataset)
df.head()

,id,text,label,Comments
0,3504,kháng sinh dự phòng nếu đã được chỉ định trong...,"[[135, 163, DISEASE]]",[]
1,3505,ngày nay các chuyên gia tin rằng viêm nội tâm ...,"[[33, 49, DISEASE], [101, 109, CAUSE]]",[]
2,3506,hướng dẫn hiện nay khuyến cáo điều trị kháng s...,"[[129, 157, DISEASE]]",[]
3,3507,bác sĩ vẫn có thể khuyên nên dùng kháng sinh p...,"[[84, 100, TREATMENT]]",[]
4,3508,đối với hầu hết những người có thông liên thất...,"[[31, 46, DISEASE], [160, 176, DISEASE]]",[]


In [70]:
df = df.drop(columns='Comments')
df['text'] = df['text'].str.strip()

Câu có hơn 512 token

In [71]:
df['token_count'] = df['text'].astype(str).apply(lambda x: len(x.split()))
ket_qua = df[df['token_count'] > 512]
print(f"Số lượng dòng có hơn 512 token: {len(ket_qua)}")
print(ket_qua[['id', 'token_count']])
df['token_count']


Số lượng dòng có hơn 512 token: 2
         id  token_count
6972  12466          563
7005  12517          618


0       43
1       42
2       34
3       25
4       39
        ..
7744    44
7745    28
7746    28
7747    34
7748    25
Name: token_count, Length: 7749, dtype: int64

Câu chỉ có 1 label 

In [72]:
df['num_labels'] = df['label'].apply(len)
one_label_rows = df[df['num_labels'] == 1]

In [73]:
print(f"Tổng số dòng trong file: {len(df)}")
print(f"Số dòng có đúng 1 label: {len(one_label_rows)}")

Tổng số dòng trong file: 7749
Số dòng có đúng 1 label: 2880


In [74]:
if not one_label_rows.empty:
    print("\nVí dụ các dòng có 1 label:")
    print(one_label_rows[['id', 'label', 'num_labels']].head())


Ví dụ các dòng có 1 label:
     id                   label  num_labels
0  3504   [[135, 163, DISEASE]]           1
2  3506   [[129, 157, DISEASE]]           1
3  3507  [[84, 100, TREATMENT]]           1
6  3513     [[70, 85, DISEASE]]           1
7  3516     [[81, 96, DISEASE]]           1


Entity Co-occurrence

In [ ]:
import pandas as pd
from itertools import combinations

def analyze_re_potential(row):
    labels = row['label']
    num_entities = len(labels)
    if num_entities < 2:
        return {
            "valid_for_re": False,
            "entity_count": num_entities,
            "pairs": []
        }
    
    entity_types = [lbl[2] for lbl in labels]
    pairs = list(combinations(entity_types, 2))
    
    return {
        "valid_for_re": True,
        "entity_count": num_entities,
        "pairs": pairs
    }

df = df.reset_index(drop=True)

df = df.loc[:, ~df.columns.duplicated()]

analysis = df.apply(analyze_re_potential, axis=1, result_type='expand')
df = pd.concat([df, analysis], axis=1)

print("Đã xử lý xong! Kích thước dữ liệu hiện tại:", df.shape)

re_data = df[df['valid_for_re'] == True]
print("=== ĐÁNH GIÁ KHẢ NĂNG LÀM RELATION EXTRACTION ===")
print(f"1. Tổng số câu ban đầu: {len(df)}")
print(f"2. Số câu CÓ THỂ chứa quan hệ (>= 2 entities): {len(re_data)} ({len(re_data)/len(df)*100:.1f}%)")
print("-" * 30)

if not re_data.empty:
    all_pairs = [pair for row_pairs in re_data['pairs'] for pair in row_pairs]
    sorted_pairs = [tuple(sorted(p)) for p in all_pairs] 
    
    from collections import Counter
    pair_counts = Counter(sorted_pairs)
    
    print("3. Top các cặp thực thể xuất hiện cùng nhau (Tiềm năng quan hệ):")
    for pair, count in pair_counts.most_common(10):
        print(f"   - {pair[0]} <---> {pair[1]}: {count} lần")

    print("-" * 30)
    print("Ví dụ 1 câu đạt chuẩn RE:")
    sample = re_data.iloc[0]
    print(f"ID: {sample['id']}")
    print(f"Text: {sample['text']}")
    print(f"Labels: {sample['label']}")
    print(f"Các cặp tiềm năng: {sample['pairs']}")
else:
    print("Cảnh báo: Không có dòng nào có từ 2 thực thể trở lên!")

Đã xử lý xong! Kích thước dữ liệu hiện tại: (7749, 8)
=== ĐÁNH GIÁ KHẢ NĂNG LÀM RELATION EXTRACTION ===
1. Tổng số câu ban đầu: 7749
2. Số câu CÓ THỂ chứa quan hệ (>= 2 entities): 4869 (62.8%)
------------------------------
3. Top các cặp thực thể xuất hiện cùng nhau (Tiềm năng quan hệ):
   - DISEASE <---> DISEASE: 10640 lần
   - SYMPTOM <---> SYMPTOM: 7614 lần
   - DISEASE <---> SYMPTOM: 6162 lần
   - TREATMENT <---> TREATMENT: 3116 lần
   - DISEASE <---> TREATMENT: 2418 lần
   - DIAGNOSTIC <---> DISEASE: 2029 lần
   - CAUSE <---> DISEASE: 1926 lần
   - DIAGNOSTIC <---> DIAGNOSTIC: 1493 lần
   - CAUSE <---> CAUSE: 1277 lần
   - SYMPTOM <---> TREATMENT: 754 lần
------------------------------
Ví dụ 1 câu đạt chuẩn RE:
ID: 3505
Text: ngày nay các chuyên gia tin rằng viêm nội tâm mạc nhiều hơn với khả năng xảy ra từ việc tiếp xúc với vi trùng một cách ngẫu nhiên hơn từ một thủ thuật nha khoa hoặc phẫu thuật điển hình .
Labels: [[33, 49, 'DISEASE'], [101, 109, 'CAUSE']]
Các cặp tiềm năng: 

In [77]:
# Tìm xem dòng nào có 'pairs' bị lỗi (độ dài < 2)
def has_bad_pairs(pairs):
    if not isinstance(pairs, list): return True
    for p in pairs:
        if len(p) < 2: return True
    return False

# Lọc ra các dòng lỗi
error_rows = df[df['pairs'].apply(has_bad_pairs)]

if not error_rows.empty:
    print(f"Phát hiện {len(error_rows)} dòng có cặp quan hệ bị lỗi (không đủ 2 thực thể/cặp).")
    print("Ví dụ dòng lỗi:", error_rows['pairs'].head())
    
    # Bạn có thể muốn loại bỏ các dòng này khỏi tập dữ liệu RE
    # df = df[~df['pairs'].apply(has_bad_pairs)]
else:
    print("Dữ liệu sạch, lỗi trước đó có thể do giá trị Null.")

Dữ liệu sạch, lỗi trước đó có thể do giá trị Null.


In [ ]:
def calculate_priority_safe(pairs):
    # [An toàn 1] Nếu ô dữ liệu là NaN hoặc không phải list -> trả về 0
    if not isinstance(pairs, list):
        return 0
    
    score = 0
    for p in pairs:
        # [An toàn 2] Nếu p không đủ 2 phần tử (ví dụ chỉ có 1 nhãn) -> Bỏ qua
        if len(p) < 2:
            continue
            
        # Logic tính điểm cũ
        try:
            p_sorted = tuple(sorted(p))
            
            # Ưu tiên cao nhất cho các cặp khác loại quan trọng
            if p_sorted in [('DISEASE', 'SYMPTOM'), ('DISEASE', 'TREATMENT'), ('CAUSE', 'DISEASE')]:
                score += 3
            # Ưu tiên vừa cho các cặp chẩn đoán
            elif p_sorted in [('DIAGNOSTIC', 'DISEASE')]:
                score += 2
            # Ít ưu tiên cho cặp cùng loại (thường là liệt kê)
            elif p[0] == p[1]:
                score += 0.5 
            else:
                score += 1
        except Exception:
            continue # Bỏ qua nếu có lỗi bất ngờ khác
            
    return score

df['priority_score'] = df['pairs'].apply(calculate_priority_safe)

print("Đã tính điểm xong mà không bị lỗi!")

Đã tính điểm xong mà không bị lỗi!


In [82]:
top_data = df.sort_values(by='priority_score', ascending=False).head(1000).copy()

print(f"Đã chọn được {len(top_data)} câu.")

Đã chọn được 1000 câu.


In [123]:
top_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 7005 to 899
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1000 non-null   int64  
 1   text            1000 non-null   object 
 2   label           1000 non-null   object 
 3   token_count     1000 non-null   int64  
 4   num_labels      1000 non-null   int64  
 5   valid_for_re    1000 non-null   bool   
 6   entity_count    1000 non-null   int64  
 7   pairs           1000 non-null   object 
 8   priority_score  1000 non-null   float64
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 71.3+ KB


In [ ]:
top_data.head() 

,id,text,label,token_count,num_labels,valid_for_re,entity_count,pairs,priority_score
7005,12517,trong đó các nguyên nhân thường gặp gồm có : c...,"[[67, 91, DISEASE], [94, 102, DISEASE], [105, ...",618,61,True,61,"[(DISEASE, DISEASE), (DISEASE, DISEASE), (DISE...",2730.0
7568,13262,nguyên nhân bệnh thiếu máu do thiếu vitamin b1...,"[[12, 26, DISEASE], [30, 47, CAUSE], [95, 112,...",470,32,True,32,"[(DISEASE, CAUSE), (DISEASE, CAUSE), (DISEASE,...",907.5
5777,11030,tuổi : đa số bệnh nhân ung thư túi mật được ch...,"[[23, 38, DISEASE], [82, 97, DISEASE], [175, 1...",355,32,True,32,"[(DISEASE, DISEASE), (DISEASE, DISEASE), (DISE...",848.0
7370,13002,"triệu chứng sốt , sưng vùng thoát vị kèm đỏ , ...","[[12, 15, SYMPTOM], [18, 36, SYMPTOM], [54, 61...",490,29,True,29,"[(SYMPTOM, SYMPTOM), (SYMPTOM, SYMPTOM), (SYMP...",710.0
6910,12389,tuy nhiên bệnh trĩ không phải luôn luôn đi ngo...,"[[10, 18, DISEASE], [40, 55, SYMPTOM], [59, 86...",397,28,True,28,"[(DISEASE, SYMPTOM), (DISEASE, SYMPTOM), (DISE...",661.5


Human labeling...

In [125]:
import json

label_studio_data = []

for idx, row in top_data.iterrows():
    text = row['text']
    original_labels = row['label'] # Dạng [(start, end, label), ...]
    
    # 1. Tạo danh sách kết quả (các thực thể đã gán nhãn)
    results = []
    for start, end, label_type in original_labels:
        results.append({
            "from_name": "label", # Tên thẻ Labels trong config XML (quan trọng)
            "to_name": "text",    # Tên thẻ Text trong config XML
            "type": "labels",
            "value": {
                "start": int(start), # Label Studio bắt buộc int chuẩn, không phải numpy int
                "end": int(end),
                "text": text[start:end],
                "labels": [label_type]
            }
        })
    
    task = {
        "data": {
            "text": text,
            "ref_id": str(row['id']) # Lưu id gốc để sau này map lại
        }
    }
    
    # 3. Nếu có nhãn, thêm vào phần predictions (dự đoán sẵn)
    if results:
        task["predictions"] = [{
            "model_version": "v1_pre_label",
            "score": 0.5, # Giả định độ tin cậy, có thể bỏ qua
            "result": results
        }]
        
    label_studio_data.append(task)

# Xuất ra file JSON
with open('data_for_label_studio.json', 'w', encoding='utf-8') as f:
    json.dump(label_studio_data, f, ensure_ascii=False, indent=4)

print(f"Đã xuất {len(label_studio_data)} mẫu ra file 'data_for_label_studio.json'")

Đã xuất 1000 mẫu ra file 'data_for_label_studio.json'


In [ ]:
re_records = []

for idx, row in top_data.iterrows():
    text = row['text']
    labels = row['label']

    entities = []
    for start, end, label_type in labels:
        entity_text = text[start:end]
        entities.append({'start': start, 'end': end, 'type': label_type, 'text': entity_text})
    
    from itertools import combinations
    if len(entities) >= 2:
        for e1, e2 in combinations(entities, 2):
            re_records.append({
                'original_id': row['id'],
                'text': text,
                'entity_1_text': e1['text'],
                'entity_1_type': e1['type'],
                'entity_2_text': e2['text'],
                'entity_2_type': e2['type'],
                'relation_label': None
            })
re_df = pd.DataFrame(re_records)

re_df

,original_id,text,entity_1_text,entity_1_type,entity_2_text,entity_2_type,relation_label
0,12517,trong đó các nguyên nhân thường gặp gồm có : c...,viêm cầu thận cấp và mạn,DISEASE,sỏi thận,DISEASE,None
1,12517,trong đó các nguyên nhân thường gặp gồm có : c...,viêm cầu thận cấp và mạn,DISEASE,hẹp động mạch thận,DISEASE,None
2,12517,trong đó các nguyên nhân thường gặp gồm có : c...,viêm cầu thận cấp và mạn,DISEASE,u tủy thượng thận,DISEASE,None
3,12517,trong đó các nguyên nhân thường gặp gồm có : c...,viêm cầu thận cấp và mạn,DISEASE,cường aldosteron,DISEASE,None
4,12517,trong đó các nguyên nhân thường gặp gồm có : c...,viêm cầu thận cấp và mạn,DISEASE,cushing,DISEASE,None
...,...,...,...,...,...,...,...
28827,4784,loại thuốc octreotide gọi là ( sandostatin ) t...,thuốc octreotide,TREATMENT,liệu pháp nội soi,TREATMENT,None
28828,4784,loại thuốc octreotide gọi là ( sandostatin ) t...,thuốc octreotide,TREATMENT,chảy máu tĩnh mạch thực quản,DISEASE,None
28829,4784,loại thuốc octreotide gọi là ( sandostatin ) t...,sandostatin,TREATMENT,liệu pháp nội soi,TREATMENT,None
28830,4784,loại thuốc octreotide gọi là ( sandostatin ) t...,sandostatin,TREATMENT,chảy máu tĩnh mạch thực quản,DISEASE,None
